In [14]:
from bs4 import BeautifulSoup
import requests
import time
from random import randint
from requests.exceptions import RequestException

In [15]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time

# Set up Selenium with headless Chrome for efficiency
options = webdriver.ChromeOptions()
options.add_argument("--headless")
options.add_argument("--disable-gpu")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/91.0.4472.124")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

try:
    # URL for CoinDesk news
    url = "https://www.coindesk.com/"
    driver.get(url)

    # Wait for headlines to load (adjust timeout as needed)
    try:
        WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.TAG_NAME, "h3"))
        )
    except Exception as e:
        print(f"Timeout waiting for headlines: {str(e)}")
        driver.quit()
        exit()

    # Parse page source with BeautifulSoup
    soup = BeautifulSoup(driver.page_source, "html.parser")

    # Find news headlines (try a broader selector first)
    headlines = soup.find_all("h3")  # Fallback: all h3 tags
    # Alternative: Try a class-based selector (update after inspection)
    # headlines = soup.find_all("h3", {"class": "your-class-here"})

    # Debugging: Print number of headlines found
    print(f"Found {len(headlines)} headlines.")

    if not headlines:
        print("No headlines found. Check HTML structure or class name.")
        # Optional: Print part of the page source for debugging
        print("Sample HTML:", soup.prettify()[:1000])
        driver.quit()
        exit()

    # Extract up to 30 headlines
    for i, headline in enumerate(headlines[:30], 1):
        try:
            title = headline.text.strip()
            # Find the parent link (if it exists)
            parent_link = headline.find_parent("a")
            link = parent_link["href"] if parent_link and parent_link.get("href") else "No link found"
            # Handle relative links
            full_link = url + link if link.startswith("/") else link
            print(f"Headline {i}: {title}\nLink: {full_link}\n")
        except Exception as e:
            print(f"Error processing headline {i}: {str(e)}")

finally:
    # Clean up
    driver.quit()

Found 58 headlines.
Headline 1: Propy Targets Legacy Title Firms in $100M Bid to Bring Real Estate On-Chain
Link: No link found

Headline 2: Featured Stories
Link: No link found

Headline 3: Dogecoin Hits $0.20 as Breakout Volume Triples Average, Confirms Bullish Setup
Link: https://www.coindesk.com//markets/2025/10/25/dogecoin-hits-usd0-20-as-breakout-volume-triples-average-confirms-bullish-setup

Headline 4: Tether Eyes Fresh Investments to Push USAT Stablecoin to 100M Americans at December Launch
Link: https://www.coindesk.com//business/2025/10/24/tether-eyes-fresh-investments-to-push-usat-stablecoin-to-100m-americans-at-december-launch

Headline 5: Inverse Head-and-Shoulders Breakout Puts XRP on Track for $2.80 Test
Link: https://www.coindesk.com//markets/2025/10/25/inverse-head-and-shoulders-breakout-puts-xrp-on-track-for-usd2-80-test

Headline 6: Dogecoin Hits $0.20 as Breakout Volume Triples Average, Confirms Bullish Setup
Link: https://www.coindesk.com//markets/2025/10/25/dogec

In [1]:
coingecko_url = 'https://api.coingecko.com/api/v3'

In [12]:
import os
import requests
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Get API key from environment
API_KEY = os.getenv("COINGECKO_API_KEY")

if not API_KEY:
    raise ValueError("🚨 Missing API key! Make sure COINGECKO_API_KEY is set in your .env file.")

# API endpoint
url = "https://pro-api.coingecko.com/api/v3/coins/top_gainers_losers"

headers = {
    "x-cg-pro-api-key": API_KEY,
    "accept": "application/json"
}

params = {
    "vs_currency": "usd"  # or another currency like "eur"
}

# Fetch data
response = requests.get(url, headers=headers, params=params)

if response.status_code != 200:
    print("Status code:", response.status_code)
    print("Response:", response.text)
    response.raise_for_status()

data = response.json()
print(data)

{'top_gainers': [{'id': 'giggle-fund', 'symbol': 'GIGGLE', 'name': 'Giggle Fund', 'image': 'https://coin-images.coingecko.com/coins/images/69414/original/giggle-fund.jpg?1758541887', 'market_cap_rank': 303, 'usd': 218.74890444050448, 'usd_24h_vol': 308240067.7960697, 'usd_24h_change': 139.06450222014035}, {'id': 'ping', 'symbol': 'PING', 'name': 'Ping', 'image': 'https://coin-images.coingecko.com/coins/images/70265/original/ping-logo.jpg?1761320085', 'market_cap_rank': 613, 'usd': 0.07314284009978662, 'usd_24h_vol': 76576940.59418456, 'usd_24h_change': 114.68221352790951}, {'id': 'game-by-virtuals', 'symbol': 'game', 'name': 'GAME by Virtuals', 'image': 'https://coin-images.coingecko.com/coins/images/51063/original/Gaming_Agent_1fe70d54ba.jpg?1729925539', 'market_cap_rank': 952, 'usd': 0.03698464020026201, 'usd_24h_vol': 12934233.232637724, 'usd_24h_change': 109.20657886469726}, {'id': 'openeden', 'symbol': 'eden', 'name': 'OpenEden', 'image': 'https://coin-images.coingecko.com/coins/i

In [13]:
from pprint import pprint
pprint(data)


{'top_gainers': [{'id': 'giggle-fund',
                  'image': 'https://coin-images.coingecko.com/coins/images/69414/original/giggle-fund.jpg?1758541887',
                  'market_cap_rank': 303,
                  'name': 'Giggle Fund',
                  'symbol': 'GIGGLE',
                  'usd': 218.74890444050448,
                  'usd_24h_change': 139.06450222014035,
                  'usd_24h_vol': 308240067.7960697},
                 {'id': 'ping',
                  'image': 'https://coin-images.coingecko.com/coins/images/70265/original/ping-logo.jpg?1761320085',
                  'market_cap_rank': 613,
                  'name': 'Ping',
                  'symbol': 'PING',
                  'usd': 0.07314284009978662,
                  'usd_24h_change': 114.68221352790951,
                  'usd_24h_vol': 76576940.59418456},
                 {'id': 'game-by-virtuals',
                  'image': 'https://coin-images.coingecko.com/coins/images/51063/original/Gaming_Agent_1fe7